In [0]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import KFold

# fix random seed for reproducibility
np.random.seed(7)

# import dataset and transform it to a numpy array
dataset = pd.read_pickle('fullDataFrame')
dataset = dataset.replace(r'\\n',' ', regex=True)
dataset['label'] = dataset['label'].astype(int)
#print(dataset)
dataset = dataset.values

# split dataset in X (input) and Y (labels)
X = dataset[:, 0:36]
Y = dataset[:, 37]
Y = Y.astype(int)


# Feature Scaling (all values between 0.0 and 1.0)
sc = StandardScaler(with_mean=True, with_std=True)
X = sc.fit_transform(X)
X = np.nan_to_num(X)

# one hot encoding of labels
onehot_encoder = OneHotEncoder(sparse=False)
Y = Y.reshape(len(Y), 1)
Y = onehot_encoder.fit_transform(Y)

# split in train / test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42, shuffle=False)



# Cross-validate
kf = KFold(5, shuffle=True, random_state=42)
    
oos_y = []
oos_pred = []
fold = 0

for train, test in kf.split(X):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train = X[train]
    y_train = Y[train]
    x_test = X[test]
    y_test = Y[test]
    
    #kernel_regularizer=regularizers.l2(0.01),
    
    # create model
    classes=3
    model = Sequential()
    model.add(Dense(36, input_dim=36, activation='relu'))
    model.add(Dense(18, activation='relu'))
    model.add(Dense(9, activation='relu'))
    model.add(Dense(classes, activation='softmax'))  # we use Softmax, so the outputs represent accuracies that add up to 1


    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=1,epochs=100,batch_size=10,shuffle=True)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)
    oos_pred.append(pred)        

    # Measure this fold's accuracy
    y_compare = np.argmax(y_test,axis=1) # For accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print(f"Fold score (accuracy): {score}")


# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
oos_y_compare = np.argmax(oos_y,axis=1) # For accuracy calculation

score = metrics.accuracy_score(oos_y_compare, oos_pred)
print(f"Final score (accuracy): {score}") 









/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Fold #1
Train on 5537 samples, validate on 1385 samples
Epoch 1/100
5537/5537 [==============================] - 2s 436us/step - loss: 0.8275 - acc: 0.6402 - val_loss: 0.7292 - val_acc: 0.7011
Epoch 2/100
5537/5537 [==============================] - 2s 343us/step - loss: 0.6493 - acc: 0.7379 - val_loss: 0.6206 - val_acc: 0.7502
Epoch 3/100
5537/5537 [==============================] - 2s 345us/step - loss: 0.5660 - acc: 0.7818 - val_loss: 0.5613 - val_acc: 0.7711
Epoch 4/100
5537/5537 [==============================] - 2s 345us/step - loss: 0.5227 - acc: 0.7992 - val_loss: 0.5553 - val_acc: 0.7588
Epoch 5/100
5537/5537 [==============================] - 2s 345us/step - loss: 0.4924 - acc: 0.8095 - val_loss: 0.5310 - val_acc: 0.7935
Epoch 6/100
5537/5537 [==============================] - 2s 345us/step - loss: 0.4701 - acc: 0.8163 - val_loss: 0.4823 - val_acc: 0.8043
Epoch 7/100
5537/5537 [==============================] - 2s 350us/step - loss: 0.4526 - acc: 0.8239 - val_loss: 0.4927 - v